# Example application: estimate the area inside a circle without calculus

Inspired by David Biersach's excellent [tutorial on Monte Carlo methods](https://github.com/dbiersach/scicomp101/tree/a863ea0cb3a02165abf681563de95dc506b47f1f/Session%2012%20-%20Monte%20Carlo%20Methods) (as well as plenty of other examples of this on the internet), we seek to estimate the area inside a circle without knowledge of calculus or the formula for the area of a circle, $A=\pi r^2.$

We will be armed with only the following information: a circle is defined by the equation $x^2 + y^2 = r^2.$

How can we proceed to do this?

First, lets make a plot of a circle inscribed within a square. We'll see why we're doing this later. Lets import some required plotting libraries and set a few defaults to make visualizing easier.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
labelsize = 12
mpl.rcParams['figure.dpi'] = 250
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
mpl.rcParams['text.usetex'] = True
plt.rc('xtick', labelsize=labelsize)
plt.rc('ytick', labelsize=labelsize)
plt.rc('axes', labelsize=labelsize)

Define a few helper functions.

In [ ]:
def make_circle(ax):
    circle_radius = 1
    circle = plt.Circle((0, 0), circle_radius, color='blue', fill=False)
    ax.add_patch(circle)
    
def set_axes_properties(ax):
    # Set the limits of the plot
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)

    # Set the aspect of the plot to be equal
    ax.set_aspect('equal')

    # Add labels and title
    ax.set_xlabel("$x$")
    ax.set_ylabel("$y$")
    
    # Add ticks manually
    ax.set_yticks([-1, 0, 1])
    ax.set_xticks([-1, 0, 1])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(1, 1))
make_circle(ax)
set_axes_properties(ax)
plt.show()

The objective of this notebook will be to estimate the area contained inside of the blue circle **without** using the equation $A=\pi r^2.$

## Monte Carlo

Monte Carlo (MC) derives its name from the Monte Carlo Casino in Monaco. The fundamental premise of MC is to use randomness to solve problems. There are too many details to go into here in this notebook, so we'll leave the theory of MC to other texts. The main point here is to show how to use NumPy to draw many random samples, and to use those samples to make an estimate for the area of the circle above. The key points to keep in mind are as follows.

Imagine you throw many darts at the picture above. The darts are always constrained to land within the square, but might not necessarily land within the circle. It stands to reason that given enough dart throws, that the ratio of the areas of the circle to the square should be equal to the ratio of the darts thrown in the circle, to the total number of darts thrown. Mathematically:

$$ \frac{A_\mathrm{circle}}{A_\mathrm{square}} = \frac{N_\mathrm{in}}{N}.$$

Thsi means that the area of the circle is simply $4\frac{N_\mathrm{in}}{N}$, since the area of the rectangle is 4. At this point, all we have to do is sample some random points on the domain of the square, and evaluate how many satisfy the equation $x^2 + y^2 < r^2,$ which would indicate a point should be added to the $N_\mathrm{in}$ counter.

In [ ]:
import numpy as np

In [ ]:
np.random.seed(123)

# Take N random points
# N should be a reasonably large number
N = int(1e5)

# Sample points in the 2d square between -1 and 1. Note the scaling and shifting factors due
# to the domain of np.random.random being [0, 1).
X = np.random.random(size=(N, 2)) * 2 - 1

`X` represents an `(N, 2)` array where the first column is $x$ and the second column is $y.$ Lets figure out which points fall within the circle with a simple boolean comparison.

In [ ]:
circle_radius = 1.0
in_circle = (X**2).sum(axis=1) < circle_radius**2  # make sure you understand this line!
where = np.where(in_circle)[0]

`np.where` extracts the indexes upon which the condition is true. This means that $N_\mathrm{in}$ is actually `len(where)` above. As such, we now have all the information to compute the area of the circle. We know from the analytic result that in fact, $A = \pi,$ since $r=1.$

In [ ]:
4.0 * len(where) / N  # looks pretty close 

Let's now visualize these points.

In [ ]:
where_not = np.where(~in_circle)[0]  # What am I doing here? Figure it out!

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(2, 2))
make_circle(ax)
set_axes_properties(ax)
size = 0.01
alpha = 0.5
ax.scatter(X[where, 0], X[where, 1], s=size, alpha=alpha, color="blue")
ax.scatter(X[where_not, 0], X[where_not, 1], s=size, alpha=alpha, color="red")
plt.show()

Above, blue points are in the circle, and red points are out of the circle. 

## Convergence of Monte Carlo methods

Can we systematically test how close our approximation is? The real question is how fast does the approximated result from MC approach the true result? We can use a basic evaluation metric, the relative error, to test this as a function of $N$.

In [ ]:
def relative_error(N, seed=123):
    """Calculates the relative error of the Monte Carlo approximation with respect to
    the true answer."""
    
    circle_radius = 1.0
    
    if seed is not None:
        np.random.seed(seed)
        
    X = np.random.random(size=(N, 2)) * 2 - 1
    
    in_circle = (X**2).sum(axis=1) < circle_radius**2
    where = np.where(in_circle)[0]
    
    approximation = 4.0 * len(where) / N
    
    return np.abs(approximation - np.pi) / np.pi

In [ ]:
N_values = 10**np.arange(1, 9)

In [ ]:
rel_errors = np.array([relative_error(n) for n in N_values])

We can plot these relative errors as a function of $N$ and figure out how the error "scales" with the number of sampled points.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 3))

ax.plot(N_values, rel_errors, "ko-")

log_grid = np.logspace(np.log10(N_values.min()), np.log10(N_values.max()), 100)
ax.plot(log_grid, 1.0 / np.sqrt(log_grid), linestyle="--", color="red")

ax.set_yscale("log")
ax.set_xscale("log")

ax.set_ylabel("Relative Error")
ax.set_xlabel("$N$")

plt.show()

It turns out that the error in all MC methods scales like $1/\sqrt{N}.$